In [26]:
import re
import pandas as pd
from bs4 import BeautifulSoup

In [11]:
file_path = r"C:\Users\Mayank\Desktop\project\financial-insights-from-sec-filings\sec-edgar-filings\AAPL\10-K\0000320193-17-000070\full-submission.txt"
with open(file_path, 'r') as file:
    file_content = file.read()

type(file_content)

str

In [12]:
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')

In [13]:
doc_start_is = [x.end() for x in doc_start_pattern.finditer(file_content)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(file_content)]

doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(file_content)]
doc_types

['10-K',
 'EX-10.8',
 'EX-10.20',
 'EX-10.21',
 'EX-12.1',
 'EX-21.1',
 'EX-23.1',
 'EX-31.1',
 'EX-31.2',
 'EX-32.1',
 'EX-101.INS',
 'EX-101.SCH',
 'EX-101.CAL',
 'EX-101.DEF',
 'EX-101.LAB',
 'EX-101.PRE',
 'GRAPHIC',
 'GRAPHIC',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'XML',
 'EXCEL',
 'XML',
 'XML',
 'XML',
 'ZIP']

In [16]:
document = {}

for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        document[doc_type] = file_content[doc_start:doc_end]

document['10-K'][0:500]

'\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>a10-k20179302017.htm\n<DESCRIPTION>10-K\n<TEXT>\n<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n\t<head>\n\t\t<!-- Document created using Wdesk 1 -->\n\t\t<!-- Copyright 2017 Workiva -->\n\t\t<title>Document</title>\n\t</head>\n\t<body style="font-family:Times New Roman;font-size:10pt;">\n<div><a name="sF73B6A857F525CF0B456675831EC2BB5"></a></div><div></div><div><br></div><div style="line-height:120%;text-align:center'

In [17]:
regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|1B|7A|7|8)\.{0,1})|(ITEM\s(1A|1B|7A|7|8))')

matches = regex.finditer(document['10-K'])

for match in matches:
    print(match)

<re.Match object; span=(37821, 37830), match='>Item 1A.'>
<re.Match object; span=(38844, 38853), match='>Item 1B.'>
<re.Match object; span=(45612, 45620), match='>Item 7.'>
<re.Match object; span=(46739, 46748), match='>Item 7A.'>
<re.Match object; span=(47809, 47817), match='>Item 8.'>
<re.Match object; span=(123704, 123713), match='>Item 1A.'>
<re.Match object; span=(196863, 196872), match='>Item 1B.'>
<re.Match object; span=(364642, 364650), match='>Item 7.'>
<re.Match object; span=(792204, 792213), match='>Item 7A.'>
<re.Match object; span=(804000, 804008), match='>Item 8.'>


In [20]:
matches = regex.finditer(document['10-K'])

test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

test_df.head()

,item,start,end
0,>item 1a.,37821,37830
1,>item 1b.,38844,38853
2,>item 7.,45612,45620
3,>item 7a.,46739,46748
4,>item 8.,47809,47817


In [21]:
test_df.replace('&#160;',' ',regex=True,inplace=True)
test_df.replace('&nbsp;',' ',regex=True,inplace=True)
test_df.replace(' ','',regex=True,inplace=True)
test_df.replace('\.','',regex=True,inplace=True)
test_df.replace('>','',regex=True,inplace=True)

test_df.head()

,item,start,end
0,item1a,37821,37830
1,item1b,38844,38853
2,item7,45612,45620
3,item7a,46739,46748
4,item8,47809,47817


In [22]:
pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')
pos_dat

,item,start,end
5,item1a,123704,123713
6,item1b,196863,196872
7,item7,364642,364650
8,item7a,792204,792213
9,item8,804000,804008


In [23]:
pos_dat.set_index('item', inplace=True)

pos_dat

,start,end
item,,
item1a,123704,123713
item1b,196863,196872
item7,364642,364650
item7a,792204,792213
item8,804000,804008


In [24]:
item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['start'].loc['item1b']]

item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item7a']]

item_7a_raw = document['10-K'][pos_dat['start'].loc['item7a']:pos_dat['start'].loc['item8']]

In [33]:
item_1a_raw[0:1000]

'>Item 1A. </font></div></td><td style="vertical-align:top;"><div style="line-height:120%;text-align:justify;font-size:9pt;"><font style="font-family:Helvetica,sans-serif;font-size:9pt;font-weight:bold;">Risk Factors</font></div></td></tr></table><div style="line-height:120%;padding-top:8px;text-align:justify;font-size:9pt;"><font style="font-family:Helvetica,sans-serif;font-size:9pt;">The following discussion of risk factors contains forward-looking statements. These risk factors may be important to understanding other statements in this Form 10-K. The following information should be read in conjunction with Part II, Item&#160;7, &#8220;Management&#8217;s Discussion and Analysis of Financial Condition and Results of Operations&#8221; and the consolidated financial statements and related notes in Part II, Item&#160;8, &#8220;Financial Statements and Supplementary Data&#8221; of this Form 10-K.</font></div><div style="line-height:120%;padding-top:16px;text-align:justify;font-size:9pt;">

In [27]:
item_1a_content = BeautifulSoup(item_1a_raw, 'lxml')

In [36]:
print(item_1a_content.prettify()[0:1000])

<html>
 <body>
  <p>
   &gt;Item 1A.
  </p>
  <td style="vertical-align:top;">
   <div style="line-height:120%;text-align:justify;font-size:9pt;">
    <font style="font-family:Helvetica,sans-serif;font-size:9pt;font-weight:bold;">
     Risk Factors
    </font>
   </div>
  </td>
  <div style="line-height:120%;padding-top:8px;text-align:justify;font-size:9pt;">
   <font style="font-family:Helvetica,sans-serif;font-size:9pt;">
    The following discussion of risk factors contains forward-looking statements. These risk factors may be important to understanding other statements in this Form 10-K. The following information should be read in conjunction with Part II, Item 7, “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and the consolidated financial statements and related notes in Part II, Item 8, “Financial Statements and Supplementary Data” of this Form 10-K.
   </font>
  </div>
  <div style="line-height:120%;padding-top:16px;text-align:justify;fon

In [31]:
print(item_1a_content.get_text("\n\n"))

>Item 1A. 

Risk Factors

The following discussion of risk factors contains forward-looking statements. These risk factors may be important to understanding other statements in this Form 10-K. The following information should be read in conjunction with Part II, Item 7, “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and the consolidated financial statements and related notes in Part II, Item 8, “Financial Statements and Supplementary Data” of this Form 10-K.

The business, financial condition and operating results of the Company can be affected by a number of factors, whether currently known or unknown, including but not limited to those described below, any one or more of which could, directly or indirectly, cause the Company’s actual financial condition and operating results to vary materially from past, or from anticipated future, financial condition and operating results. Any of these factors, in whole or in part, could materially and advers